In [82]:
# !python3 -m pip install --upgrade pip
# !python3 -m pip install python-okx --upgrade

In [83]:
import okx.Account as Account
import requests
from datetime import datetime

In [84]:
# demo api key 和secret key，不要用成实盘的
def read_config(filepath="config.txt"):
    config = {}
    with open(filepath, 'r') as file:
        for line in file:
            if '=' in line:
                key, value = line.strip().split('=', 1)
                config[key.strip()] = value.strip()
    return config

config = read_config()
api_key = config.get("api_key")
secret_key = config.get("secret_key")
passphrase = config.get("passphrase")

In [85]:

# check if the api setup works
flag = "1"  # live trading: 0, demo trading: 1
accountAPI = Account.AccountAPI(api_key, secret_key, passphrase, False, flag)
result = accountAPI.get_account_balance()
print(result)

{'msg': 'Invalid OK-ACCESS-KEY', 'code': '50111'}


In [86]:
class Candle:

    def __init__(self, timestamp, open_p, high, low, close):
        self.timestamp = timestamp
        self.open_p = open_p
        self.high = high
        self.low = low
        self.close = close

    def __str__(self) -> str:
        return f"Time: {self.timestamp}, Open: {self.open_p}, High: {self.high}, Low: {self.low}, Close: {self.close}"


In [87]:
def get_4h_candlesticks(inst_id='BTC-USDT', limit=100):
    url = "https://www.okx.com/api/v5/market/candles"
    params = {
        "instId": inst_id,
        "bar": "4H",
        "limit": limit
    }
    

    candles = []
    response = requests.get(url, params=params)
    if response.status_code == 200:
        json_data = response.json()
        data = json_data['data']
        print(len(data))
        # fetched data are from most recent to oldest
        for datapoint in data:
            # last element indicate if the current candle is finished or not
            # finish = candle[8]
            timestamp, open_p, high, low, close = datapoint[0:5]
            # print(f"Time: {timestamp}, Open: {open_p}, High: {high}, Low: {low}, Close: {close}")
            new_candle = Candle(to_readable_time(int(timestamp)), float(open_p), float(high), float(low), float(close))
            candles.append(new_candle)
        candles.reverse()
        return candles
    else:
        print("Error:", response.status_code, response.text)
        return None


def to_readable_time(timestamp_ms: int) -> str:
    dt = datetime.fromtimestamp(timestamp_ms / 1000).astimezone()
    return dt.strftime('%Y-%m-%d %H:%M:%S')

candles = get_4h_candlesticks()

for candle in candles:
    print(candle)

100
Time: 2025-05-16 05:00:00, Open: 103672.0, High: 104285.7, Low: 103339.3, Close: 104187.9
Time: 2025-05-16 09:00:00, Open: 104188.0, High: 104555.0, Low: 103585.6, Close: 104023.3
Time: 2025-05-16 13:00:00, Open: 104025.5, High: 104036.0, Low: 103391.6, Close: 103461.2
Time: 2025-05-16 17:00:00, Open: 103461.2, High: 103579.8, Low: 102600.0, Close: 103505.5
Time: 2025-05-16 21:00:00, Open: 103506.5, High: 103729.1, Low: 103286.0, Close: 103450.0
Time: 2025-05-17 01:00:00, Open: 103449.9, High: 103571.3, Low: 102750.1, Close: 102998.0
Time: 2025-05-17 05:00:00, Open: 102998.0, High: 103187.9, Low: 102613.9, Close: 102950.0
Time: 2025-05-17 09:00:00, Open: 102950.1, High: 103372.5, Low: 102763.0, Close: 103153.0
Time: 2025-05-17 13:00:00, Open: 103153.0, High: 103480.0, Low: 102901.0, Close: 103133.6
Time: 2025-05-17 17:00:00, Open: 103133.7, High: 103419.4, Low: 103100.1, Close: 103282.6
Time: 2025-05-17 21:00:00, Open: 103282.6, High: 103976.3, Low: 103279.6, Close: 103927.9
Time: 

In [88]:
class Extreme:
    def __init__(self, timestamp, open_p, high, low, close, extreme_type):
        self.timestamp = timestamp
        self.open_p = open_p
        self.high = high
        self.low = low
        self.close = close
        self.type = extreme_type

    def __str__(self) -> str:
        return f"{self.type} Extreme: Time: {self.timestamp}, Open: {self.open_p}, High: {self.high}, Low: {self.low}, Close: {self.close}"


In [89]:
def find_extremes(candles: Candle):
    """
    通过k线高点高于前后各一根k线或者低于前后各一根k线来找到每个高点和低点
    除了MSB都是根据影线来判断
    """
    extremes = []
    length = len(candles)
    # TODO: 第一根k线永远不可能是极值点，这一点要不要考虑
    for i in range(1, length-1):
        is_high = candles[i].high >= candles[i-1].high and candles[i].high >= candles[i+1].high
        is_low = is_low = candles[i].low <= candles[i-1].low and candles[i].low <= candles[i+1].low

        # 高点
        if is_high and is_low:
            new_extreme = Extreme(candles[i].timestamp, candles[i].open_p, candles[i].high, candles[i].low, candles[i].close, 'both')
            extremes.append(new_extreme)
        elif is_high:
            new_extreme = Extreme(candles[i].timestamp, candles[i].open_p, candles[i].high, candles[i].low, candles[i].close, 'high')
            extremes.append(new_extreme)
        elif is_low:
            new_extreme = Extreme(candles[i].timestamp, candles[i].open_p, candles[i].high, candles[i].low, candles[i].close, 'low')
            extremes.append(new_extreme)
    return extremes



def filter_extremes(extremes):
    """
    如果出现高点后接高点的情况，记录两个高点中的最高点信息，
    如果出现低点后接低点的情况，记录两个低点中的最低点信息，
    如果一根k线既是高点又是低点，则根据之前的点来判断，如果之前的点是高点，则记录该点的低点信息，
    反之，如果之前的点是低点，则记录该点的高点信息，从而完成初步的过滤，但仍存在的问题是记录小级别高低点和趋势性暂时未知。  
    """

    length = len(extremes)

    # 如果最开始就是both，先默认最开始为high
    if extremes[0] == 'both':
        extremes[0].type = 'high'

    filtered_extremes = [extremes[0]] # 保留起始点作为趋势判断的开始

    
    for i in range(1, length-1):
        prev = filtered_extremes[-1]
        cur = extremes[i]


        # TODO：还是上面提出的问题，如果最开始就是both呢？
        if cur.type == "both":
            cur.type = "low" if prev.type == "high" else "high"

            
        if cur.type == prev.type:
            if cur.type == "high":
                # 连续高点，保留更高
                if cur.high > prev.high:
                    filtered_extremes[-1] = cur
            elif cur.type == "low":
                # 连续低点，保留更低
                if cur.low < prev.low:
                    filtered_extremes[-1] = cur
        else:
            # 不同类型极值点，直接加入
            filtered_extremes.append(cur)
        
    return filtered_extremes

extremes = find_extremes(candles)
for e in extremes:
    print(e)
print("\n-------------------------------------------\n")


filtered_extremes = filter_extremes(extremes)
for e in filtered_extremes:
    print(e)


high Extreme: Time: 2025-05-16 09:00:00, Open: 104188.0, High: 104555.0, Low: 103585.6, Close: 104023.3
low Extreme: Time: 2025-05-16 17:00:00, Open: 103461.2, High: 103579.8, Low: 102600.0, Close: 103505.5
high Extreme: Time: 2025-05-16 21:00:00, Open: 103506.5, High: 103729.1, Low: 103286.0, Close: 103450.0
low Extreme: Time: 2025-05-17 05:00:00, Open: 102998.0, High: 103187.9, Low: 102613.9, Close: 102950.0
high Extreme: Time: 2025-05-17 13:00:00, Open: 103153.0, High: 103480.0, Low: 102901.0, Close: 103133.6
low Extreme: Time: 2025-05-18 05:00:00, Open: 103803.9, High: 105666.6, Low: 103698.6, Close: 105515.9
low Extreme: Time: 2025-05-18 13:00:00, Open: 103882.4, High: 106669.0, Low: 103270.7, Close: 106468.7
high Extreme: Time: 2025-05-18 17:00:00, Open: 106468.9, High: 107140.1, Low: 103546.2, Close: 103609.4
low Extreme: Time: 2025-05-18 21:00:00, Open: 103610.0, High: 103772.0, Low: 102080.4, Close: 103222.1
low Extreme: Time: 2025-05-19 05:00:00, Open: 102960.5, High: 104969.

In [90]:
def find_initial_trend(filtered_extremes):
    """
    从记录的高低点信息中从最开始往后遍历，每次读取四个点，
    即两个高点信息和两个低点信息，当出现两个更高高点和两个更高低点时，
    记录初始趋势为上升。当出现两个更低高点和两个更低低点时，记录初始趋势为下降
    """
    idx = 0
    while idx < len(filtered_extremes) - 3:
        if filtered_extremes[idx].type == 'low' and \
            filtered_extremes[idx+1].type == 'high' and \
            filtered_extremes[idx+2].type == 'low' and \
            filtered_extremes[idx+3].type == 'high' and \
            filtered_extremes[idx].low < filtered_extremes[idx+2].low and \
            filtered_extremes[idx+1].high < filtered_extremes[idx+3].high:
                return 'rise'
        
        elif filtered_extremes[idx].type == 'high' and \
            filtered_extremes[idx+1].type == 'low' and \
            filtered_extremes[idx+2].type == 'high' and \
            filtered_extremes[idx+3].type == 'low' and \
            filtered_extremes[idx].high > filtered_extremes[idx+2].high and \
            filtered_extremes[idx+1].low > filtered_extremes[idx+3].low:
                return "drop"
        else:
            idx += 1
        
    # 没有明显趋势，震荡区间
    return "bumpy"
    # TODO: 没有讨论怎么处理？


def analyze_trend(initial_trend, filtered_extremes):
    key_points = []  # 用于记录趋势变化和关键点
    state = initial_trend  # 当前趋势
    i = 0

    # 初始化状态维护变量
    if state == 'rise':
        last_higher_low = None
        last_higher_high = None
        # 只有低点高于前一个低点，并且下一个高点被确认为higher_high时，才会记录之前的低点为higher_low
        # 暂时存起来用作后续判定
        candidate_lows = []
    else:
        last_lower_high = None
        last_lower_low = None
        candidate_highs = []

    while i < len(filtered_extremes):
        point = filtered_extremes[i]

        if state == 'rise':
            if point.type == 'low':
                if last_higher_low:
                    # 如果之前有低点，并且当前点的实体部分低于之前低点的low
                    if point.close < last_higher_low.low:
                        # 结构破坏，趋势反转为下跌
                        state = 'drop'
                        key_points.append({'trend': 'drop', 'trigger': point})
                        # 确定反转为下跌之后，就可以确定第一对lower_low和lower_high
                        last_lower_low = point
                        last_lower_high = filtered_extremes[i - 1] if i > 0 else None
                        # 此时变为下降趋势，重置candidates高点
                        candidate_highs = []
                    else:
                        # 如果之前有低点，但是没有任何跌破，说明在震荡，把当前点放在candidates low里面以便后续使用
                        # 等于不算做跌破
                        candidate_lows.append(point)
                else:
                    # 如果上涨趋势之前还没有higher low
                    candidate_lows.append(point)

            elif point.type == 'high':
                # 如果之前没有高点，或者当前点实体部分比之前高点高，趋势延续
                if last_higher_high is None or point.high > last_higher_high.high:
                    last_higher_high = point
                    if candidate_lows:
                        # 如果存在低点，则选取之前的震荡区间中的最低点作为higher_low
                        # TODO: !!!!! 这个代码逻辑需要仔细考虑 
                        last_higher_low = min(candidate_lows, key=lambda p: p.low)
                        candidate_lows.clear()

        else:  # state == 'drop'
            if point.type == 'high':
                # 存在lower_high，并且实体突破原来的lower_high
                if last_lower_high:
                    if point.close > last_lower_high.high:
                        # 结构破坏，趋势反转为上涨
                        state = 'rise'
                        key_points.append({'trend': 'rise', 'trigger': point})
                        last_higher_high = point
                        last_higher_low = filtered_extremes[i - 1] if i > 0 else None
                        candidate_lows = []
                    else:
                        candidate_highs.append(point)
                else:
                    candidate_highs.append(point)
            elif point.type == 'low':
                # 如果之前没有低点，或者当前点实体部分比之前低点低，趋势延续
                if last_lower_low is None or point.low < last_lower_low.low:
                    last_lower_low = point
                    if candidate_highs:
                        last_lower_high = max(candidate_highs, key=lambda p: p.high)
                        candidate_highs.clear()
        i += 1

    return key_points

In [91]:
initial_trend = find_initial_trend(filtered_extremes)
print(initial_trend)


structure_transitions = analyze_trend(initial_trend, filtered_extremes)
print(structure_transitions)
for change in structure_transitions:
    print(f"Trend changed to {change['trend']} at {change['trigger'].timestamp}")

rise
[{'trend': 'drop', 'trigger': <__main__.Extreme object at 0x0000028DCD4F9550>}, {'trend': 'rise', 'trigger': <__main__.Extreme object at 0x0000028DCD4FB110>}, {'trend': 'drop', 'trigger': <__main__.Extreme object at 0x0000028DCD4F99A0>}]
Trend changed to drop at 2025-05-23 13:00:00
Trend changed to rise at 2025-05-25 21:00:00
Trend changed to drop at 2025-05-29 17:00:00
